In [53]:
import datetime, os, time, pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import applications, losses, models, layers, optimizers, callbacks, utils
from tensorflow.keras.layers.experimental import preprocessing

In [54]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [69]:
data_dir = pathlib.Path("D:Dataset/랜드마크 이미지/Training/")

In [70]:
image_count = len(list(data_dir.glob('*/*.jpg'))) # 총 이미지 개수
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [71]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names, len(class_names))

['6.25격전지개미고개' 'BRT작은미술관' '가락마을20단지호반베르디움5차아파트' '가락마을3단지호반베르디움4차아파트'
 '가온마을1단지힐스테이트세종2차' '가재마을세종호반베르디움2단지아파트' '고운뜰전망대' '국립세종도서관' '국립세종수목원'
 '국립조세박물관' '금강수목원' '금강수변공원' '금강자연휴양림' '금남남산영당' '금남모인당' '금남문절사' '김종서장군묘소'
 '나성독락정' '덕성서원' '덕천군사우' '도램마을10단지호반베르디움어반시티아파트' '도램마을15단지힐스테이트아파트'
 '밀양박씨오충정려' '박산리작약골쌍탑제' '범지기마을10단지푸르지오아파트' '범지기마을7단지호반베르디움에코시티아파트'
 '범지기마을8단지푸르지오아파트' '베어트리파크' '병산사' '부강성당' '부강초등학교강당' '비암사' '새뜸1단지메이저시티푸르지오'
 '새뜸마을10단지더샵힐스테이트아파트' '새뜸마을11단지더샵힐스테이트아파트' '새뜸마을14단지더샵힐스테이트아파트'
 '새뜸마을5단지아이파크메이저아파트' '새뜸마을6단지힐스테이트메이저아파트' '성요한성당' '세종e편한세상푸르지오아파트'
 '세종고속시외버스터미널' '세종대곡리삼층석탑' '세종시립민속박물관' '세종전통장류박물관' '세종특별자치시마을기록문화관'
 '세종행복도시홍보관' '세종호수공원' '세종힐스테이트3차' '수루배마을4단지더샵예미지' '수루배마을6단지세종더샵예미지'
 '연기대첩비공원' '연기봉산동의향나무' '연기향교' '연기향토박물관' '연동송용리마애여래입상' '연화사' '영평사' '은하수공원'
 '자성사' '전의비암사극락보전' '전의비암사삼층석탑' '전의향교' '조천연꽃공원' '조치원문화정원' '조치원봉산영당'
 '조치원신흥e편한세상아파트' '조치원신흥푸르지오아파트' '조치원역경부선' '조치원자이아파트' '조치원죽림푸르지오아파트'
 '천주교대전교구세종성프란치스코성당' '첫마을4단지푸르지오아파트' '첫마을5단지푸르지오아파트' '첫마을6단지힐스테이트아파트'
 '첫마을7단지삼성래미안아파트' '초려역사공원' '충청남도산

In [72]:
val_size = int(image_count * 0.15)
train_ds = list_ds.skip(val_size) # 20% 를 넘김
valid_ds = list_ds.take(val_size) # 20% 를 가짐 
NUM_TRAIN = tf.data.experimental.cardinality(train_ds).numpy()
NUM_VALID = tf.data.experimental.cardinality(valid_ds).numpy()
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(valid_ds).numpy())

10537
1859


In [73]:
batch_size = 32
SIZE = 224
img_height = SIZE
img_width = SIZE
image_size=(img_height, img_width)

In [74]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [75]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [76]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [77]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
valid_ds = valid_ds.map(process_path, num_parallel_calls=AUTOTUNE)
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    
for image, label in valid_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  56
Image shape:  (224, 224, 3)
Label:  28


In [78]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

In [79]:
def configure_for_performance_train(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


def configure_for_performance_test(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


train_ds = configure_for_performance_train(train_ds)
valid_ds = configure_for_performance_test(valid_ds)

In [80]:
base_model = applications.EfficientNetB0(input_shape=(img_height,img_width,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False # 베이스 모델은 학습하지 않음

In [81]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(img_height, img_width, 3))
    
    if tf.data == train_ds:
        y = img_augmentation(x)
    
    y = base_model(x, training=False)

    # Rebuild
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    top_dropout_rate = 0.2
    y = layers.Dropout(top_dropout_rate, name="top_dropout")(y)
    y = layers.Dense(84, activation="softmax", name="Pred")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=0.01)
    loss = losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=["acc"])
    return model
model = build_eff_model()

In [82]:
ckp = callbacks.ModelCheckpoint(
    "model_2.h5",  # 경로
    save_best_only=True,  # 가장 높은 점수의 모델만
    monitor="val_acc")  # val_acc 를 보고 판단
early_stopping = callbacks.EarlyStopping(
    monitor="val_acc",
    patience=7)  # 7번까지는 기다릴수있다(낮아지는 횟수가 아니라 최고점 이후 점수가 상승하지 않는 횟수)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_acc",
    patience=3,
    factor=0.1  # 학습률을 1/10 으로 줄임
)

In [83]:
model.fit(
    train_ds,
    epochs=100,
    validation_data=valid_ds,
    callbacks=[ckp, early_stopping, reduce_lr]
)

Epoch 1/100
330/330 [==============================] - 116s 281ms/step - loss: 1.2639 - acc: 0.7681 - val_loss: 0.1904 - val_acc: 0.9543
Epoch 2/100
330/330 [==============================] - 19s 58ms/step - loss: 0.1420 - acc: 0.9679 - val_loss: 0.1815 - val_acc: 0.9699
Epoch 3/100
330/330 [==============================] - 19s 58ms/step - loss: 0.1629 - acc: 0.9696 - val_loss: 0.1862 - val_acc: 0.9710
Epoch 4/100
330/330 [==============================] - 19s 59ms/step - loss: 0.1524 - acc: 0.9782 - val_loss: 0.2711 - val_acc: 0.9699
Epoch 5/100
330/330 [==============================] - 19s 59ms/step - loss: 0.1097 - acc: 0.9809 - val_loss: 0.2992 - val_acc: 0.9656
Epoch 6/100
330/330 [==============================] - 19s 58ms/step - loss: 0.1356 - acc: 0.9817 - val_loss: 0.3174 - val_acc: 0.9672
Epoch 7/100
330/330 [==============================] - 19s 59ms/step - loss: 0.0755 - acc: 0.9883 - val_loss: 0.1816 - val_acc: 0.9785
Epoch 8/100
330/330 [==============================] 

* 미세조정

In [90]:
for layer in base_model.layers:
    if "Conv2D" in str(layer):
        layer.trainable = True

In [91]:
model.fit(train_ds,
          epochs=100,
          validation_data=valid_ds,
          callbacks=[ckp, early_stopping, reduce_lr])

Epoch 1/100
330/330 [==============================] - 19s 57ms/step - loss: 9.2406e-04 - acc: 0.9997 - val_loss: 0.0937 - val_acc: 0.9882
Epoch 2/100
330/330 [==============================] - 19s 57ms/step - loss: 0.0019 - acc: 0.9991 - val_loss: 0.0935 - val_acc: 0.9882
Epoch 3/100
330/330 [==============================] - 19s 58ms/step - loss: 0.0026 - acc: 0.9993 - val_loss: 0.0937 - val_acc: 0.9887
Epoch 4/100
330/330 [==============================] - 19s 58ms/step - loss: 0.0016 - acc: 0.9993 - val_loss: 0.0930 - val_acc: 0.9887
Epoch 5/100
330/330 [==============================] - 19s 58ms/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0929 - val_acc: 0.9887
Epoch 6/100
330/330 [==============================] - 19s 58ms/step - loss: 0.0036 - acc: 0.9991 - val_loss: 0.0934 - val_acc: 0.9887
Epoch 7/100
330/330 [==============================] - 19s 58ms/step - loss: 0.0042 - acc: 0.9989 - val_loss: 0.0936 - val_acc: 0.9882
Epoch 8/100
330/330 [==============================